# Importando os codigos

In [1]:
from preprocess import pre_processing
from teste_estacionaridade import teste_estacionaridade
from diferenciacao import diferenciacao, invert_diferenciacao
from predicao import forecast
from validacao import acuracia
from sep_treinoteste import split
from validacao import acuracia, rmse, mape


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Importando o df original
original = pd.read_csv('static\\uploads\\VendasBrutasNormalizadas_v5.csv', parse_dates=['DATA']).sort_values(ascending = True, by = 'DATA')

In [3]:
# Importando as projeções 60 meses
projecao_60 = pd.read_csv('static\downloads\projecao.txt', sep = ',', parse_dates=['Unnamed: 0'])
projecao_60.rename(columns = {'Unnamed: 0': 'data'}, inplace= True)

# Importando as projeções acima de 48 meses
projecao_48 = pd.read_csv('static\downloads\projecao2.txt', sep = ',', parse_dates=['Unnamed: 0'])
projecao_48.rename(columns = {'Unnamed: 0': 'data'}, inplace= True)

#Importando as projeções acima de 36 meses
projecao_36 = pd.read_csv('static\downloads\projecao3.txt', sep = ',', parse_dates=['Unnamed: 0'])
projecao_36.rename(columns = {'Unnamed: 0': 'data'}, inplace= True)

# Importando as projeções acima de 24 meses
projecao_24 = pd.read_csv('static\downloads\projecao4.txt', sep = ',', parse_dates=['Unnamed: 0'])
projecao_24.rename(columns = {'Unnamed: 0': 'data'}, inplace= True)


In [6]:
# Capturando só o nescessário das previsões
ano = '2019'

to_append60 = pd.melt(projecao_60, id_vars='data', value_vars=projecao_60.columns.tolist()[1:], value_name = 'predicao')
to_append60 = to_append60[to_append60['data'].dt.strftime('%Y') == ano]
to_append60.reset_index(inplace= True, drop=True)

to_append48 = pd.melt(projecao_48, id_vars='data', value_vars=projecao_48.columns.tolist()[1:], value_name = 'predicao')
to_append48 = to_append48[to_append48['data'].dt.strftime('%Y') == ano]
to_append48.reset_index(inplace= True, drop=True)

to_append36 = pd.melt(projecao_36, id_vars='data', value_vars=projecao_36.columns.tolist()[1:], value_name = 'predicao')
to_append36 = to_append36[to_append36['data'].dt.strftime('%Y') == ano]
to_append36.reset_index(inplace= True, drop=True)

to_append24 = pd.melt(projecao_24, id_vars='data', value_vars=projecao_24.columns.tolist()[1:], value_name = 'predicao')
to_append24 = to_append24[to_append24['data'].dt.strftime('%Y') == ano]
to_append24.reset_index(inplace= True, drop=True)

In [7]:
to_append60

,data,variable,predicao
0,2019-01-01,00037shniteroiplaza,6.625497e+06
1,2019-02-01,00037shniteroiplaza,4.366585e+06
2,2019-03-01,00037shniteroiplaza,6.605543e+06
3,2019-04-01,00037shniteroiplaza,5.541524e+06
4,2019-05-01,00037shniteroiplaza,6.691704e+06
...,...,...,...
2119,2019-08-01,00183shpatiomixresende,9.359211e+05
2120,2019-09-01,00183shpatiomixresende,9.845140e+05
2121,2019-10-01,00183shpatiomixresende,7.652823e+05
2122,2019-11-01,00183shpatiomixresende,9.930892e+05


In [24]:
appends = [to_append60, to_append48, to_append36, to_append24 ]
for to_append in appends: 

    row_df = pd.DataFrame([])

    for j in range(len(to_append)):

        for i in range(len(res_list)):

            if (to_append.iloc[j,1] == res_list[i]['id']):
                    print(to_append.iloc[j,1])

6shiguatemimaceio
00096shiguatemimaceio
00096shiguatemimaceio
00096shiguatemimaceio
00096shiguatemimaceio
00096shiguatemimaceio
00047shparkbrasilia
00047shparkbrasilia
00047shparkbrasilia
00047shparkbrasilia
00047shparkbrasilia
00047shparkbrasilia
00047shparkbrasilia
00047shparkbrasilia
00047shparkbrasilia
00047shparkbrasilia
00047shparkbrasilia
00047shparkbrasilia
00115shgranderio
00115shgranderio
00115shgranderio
00115shgranderio
00115shgranderio
00115shgranderio
00115shgranderio
00115shgranderio
00115shgranderio
00115shgranderio
00115shgranderio
00115shgranderio
00087shrecife
00087shrecife
00087shrecife
00087shrecife
00087shrecife
00087shrecife
00087shrecife
00087shrecife
00087shrecife
00087shrecife
00087shrecife
00087shrecife
00100shamazonas
00100shamazonas
00100shamazonas
00100shamazonas
00100shamazonas
00100shamazonas
00100shamazonas
00100shamazonas
00100shamazonas
00100shamazonas
00100shamazonas
00100shamazonas
00235ljcentrodemanaus
00235ljcentrodemanaus
00235ljcentrodemanaus
00

In [6]:
#Criando os ids  das lojas com os nomes dos responssáveis
original['id']     = original["NOME_LOJA_COMPLETO"].str.lower()

for i in range(len(original.NOME_LOJA_COMPLETO)):  
    original.id[i] = original['id'][i].replace(" ", "")
    original.id[i] = original['id'][i].replace("-", "")

#Criando um dicionário com as principais informações do dataframe original
original_dict = original[['NOME_LOJA_COMPLETO', 'CODIGO_LOJA', 'NOME_LOJA', 'REGIONAL',
       'GGL', 'GERENTE_N1', 'GERENTE_N2', 'CIDADE', 'ESTADO', 'UF',  'id']].to_dict('records')

res_list = []
for i in range(len(original_dict)):
    if original_dict[i] not in original_dict[i + 1:]:
        res_list.append(original_dict[i])

In [7]:
# Criando um looping de iterações para que cada Id com sua respctiva data tenha uma linha no df novo com as  informações faltantes
dados_graficos = original
appends = [to_append60, to_append48, to_append36, to_append24 ]
for to_append in appends: 

    row_df = pd.DataFrame([])

    for j in range(len(to_append)):

        for i in range(len(res_list)):

            if (to_append.iloc[j,1] == res_list[i]['id']):

                row = pd.Series([to_append.iloc[j,0], to_append.iloc[j,1], to_append.iloc[j,2], res_list[i]['NOME_LOJA_COMPLETO'], res_list[i]['CODIGO_LOJA'], res_list[i]['REGIONAL'], res_list[i]['GGL'], res_list[i]['GERENTE_N1'], res_list[i]['GERENTE_N2'], res_list[i]['CIDADE'], res_list[i]['ESTADO'], res_list[i]['UF']])

                row_df = row_df.append(row, ignore_index=True)

    row_df.columns = ['data', 'variable', 'predicao', 'NOME_LOJA_COMPLETO', 'CODIGO_LOJA', 'REGIONAL', 'GGL', 'GERENTE_N1', 'GERENTE_N2',  'CIDADE', 'ESTADO', 'UF']

    to_apend = row_df

    # Dropando as colunas não necessárias
    to_apend = to_apend.drop(['variable'], axis = 1)

    # Renomeando as colunas no dataframe a ser apendado
    to_apend.rename(columns={'data': 'DATA', 'predicao': 'VENDA_BRUTA'}, inplace = True)

    # Criando um dataframe com o nome das colunas do dataframe original
    colunas = original.columns

    # Inserindo as colunas faltantes para ficar igual ao dataframe original

    to_apend['ORC_REAL']='PREDITO'
    to_apend['PERIODO']='tbd'
    to_apend['ANO'] = to_apend['DATA'].dt.year
    to_apend['MES'] = to_apend['DATA'].dt.month
    to_apend['DIA'] = to_apend['DATA'].dt.day
    to_apend['ANO-MES'] = 'tbd'
    to_apend['NOME_LOJA'] = to_apend['NOME_LOJA_COMPLETO']
    to_apend['MODELO']='tbd'
    to_apend['DESEMPENHO']='tbd'

    to_apend = to_apend.reindex(columns=colunas)

    dados_graficos = pd.concat([dados_graficos, to_apend])

In [8]:
sum = 0
for to_append in appends:
    sum = sum + len(to_append.variable.unique())
    print(sum, len(to_append.variable.unique()))

177 177
238 61
262 24
288 26


In [9]:
len(dados_graficos.loc[dados_graficos.ORC_REAL == 'PREDITO','NOME_LOJA_COMPLETO'].unique())

288

In [10]:
lojas_pred = dados_graficos.loc[(dados_graficos.ORC_REAL == 'PREDITO', 'NOME_LOJA_COMPLETO')].unique()

arquivador_acu = pd.DataFrame(columns = ['ERRO', 'MAE', 'MFE', 'MSE', 'RMSE', 'MAPE'])

#
for loja in lojas_pred:
    orcado  = dados_graficos.loc[(dados_graficos.NOME_LOJA_COMPLETO == loja)&(dados_graficos.ORC_REAL == "ORCADO")][['DATA','VENDA_BRUTA','NOME_LOJA_COMPLETO']].set_index('DATA')
    
    predito = dados_graficos.loc[(dados_graficos.NOME_LOJA_COMPLETO == loja)&(dados_graficos.ORC_REAL == "PREDITO")][['DATA','VENDA_BRUTA','NOME_LOJA_COMPLETO']].set_index('DATA')
    fodase = acuracia(orcado.VENDA_BRUTA, predito.VENDA_BRUTA)
    fodase.drop(columns = ['Métricas'], inplace = True)
    fodase = fodase.T
    fodase.columns = ['ERRO', 'MAE', 'MFE', 'MSE', 'RMSE', 'MAPE']
    fodase.reset_index(drop = True, inplace = True)
    fodase['NOME_LOJA_COMPLETO'] = orcado.NOME_LOJA_COMPLETO.drop_duplicates().reset_index(drop = True)
    arquivador_acu = pd.concat([arquivador_acu, fodase])
arquivador_acu.reset_index(drop = True, inplace = True)
arquivador_acu.to_csv('static\downloads\erros.csv')

In [18]:
dados_graficos = pd.merge(left=dados_graficos, right=arquivador_acu[['MAPE', 'NOME_LOJA_COMPLETO']], how='left', left_on='NOME_LOJA_COMPLETO', right_on='NOME_LOJA_COMPLETO')

In [20]:
#dados_graficos.drop(columns='id', inplace=True)
dados_graficos.to_csv('DadosGraficos_v1.csv')

In [ ]:
#list(dict.fromkeys(sorted(original.NOME_LOJA_COMPLETO.to_list())))
#list(dict.fromkeys(sorted(to_append.variable.to_list())))
#projecao_60.set_index('data', inplace = True)
#projecao_60 = projecao_60['2019'].reset_index()
#projecao_60

#original[[['NOME_LOJA_COMPLETO', 'ORC_REAL', 'PERIODO', 'VENDA_BRUTA', 'ANO', 'MES', 'DIA', 'ANO-MES', 'DATA', 'CODIGO_LOJA', 'NOME_LOJA', 'REGIONAL', 'GGL', 'GERENTE_N1', 'GERENTE_N2', 'CIDADE', 'ESTADO', 'UF', 'MODELO', 'DESEMPENHO', 'id']]